# SIDA: Speaker Identification for Archives

In [ ]:
import attk
import os
import csv
import numpy as np
import librosa
import timeit
import random
import subprocess
import urllib2
from sklearn.externals import joblib
from numpy import ma
from aubio import source, pitch
from moviepy.audio.io import AudioFileClip
from IPython.display import display, Audio
import pandas as pd

!mkdir -p /sharedfolder/applause_classifier/

os.chdir('/sharedfolder/applause_classifier/')

training_audio_dir_name = "Applause"

class_dir_pathname = '/sharedfolder/applause_classifier/_classes_' + training_audio_dir_name

subprocess.call(['mkdir', '-p', class_dir_pathname])

In [ ]:
## Downloading male and female training data
# (Uncomment the commands below first.)

os.chdir('/sharedfolder/applause_classifier/')

#!wget -N http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/females_5k.zip
#!wget -N http://xtra.arloproject.com/datasets/audio/Mozilla_CV_UBM_Subset/males_5k.zip
#!wget -N http://xtra.arloproject.com/datasets/aapb-ubm/Male_AAPB_171110.zip
#!wget -N http://xtra.arloproject.com/datasets/aapb-ubm/Female_AAPB_171110.zip

#!unzip -o females_5k.zip
#!unzip -o males_5k.zip
#!unzip -o Male_AAPB_171110.zip
#!unzip -o Female_AAPB_171110.zip

In [ ]:
## Downloading applause training data
# (Uncomment the command below first.)

#!git clone https://github.com/hipstas/applause-classifier.git

In [ ]:
## Creating directories for our two classes

applause_path = os.path.join(class_dir_pathname, 'Applause')
non_applause_path = os.path.join(class_dir_pathname, 'Non-Applause')

subprocess.call(['mkdir', applause_path])
subprocess.call(['mkdir', non_applause_path])

In [ ]:
## Consolidating all non-applause clips in a single directory

!mv females_5k/* '/sharedfolder/applause_classifier/_classes_Applause/Non-Applause'
!mv males_5k/* '/sharedfolder/applause_classifier/_classes_Applause/Non-Applause'
!mv Male_AAPB_171110/* '/sharedfolder/applause_classifier/_classes_Applause/Non-Applause'
!mv Female_AAPB_171110/* '/sharedfolder/applause_classifier/_classes_Applause/Non-Applause'

In [ ]:
os.chdir('/sharedfolder/applause_classifier/applause-classifier')

!unzip -o *.zip

In [ ]:
## Unzipping applause training data

!unzip -o Applause_from_Bill_Clinton_speeches_1.zip
!unzip -o Applause_from_Bill_Clinton_speeches_2.zip
!unzip -o applause_pt1.zip
!unzip -o applause_pt2.zip

In [ ]:
## Moving all applause WAV clips to the Applause class directory

!mv */*.wav '/sharedfolder/applause_classifier/_classes_Applause/Applause'

In [ ]:
%%capture
## Extracting features (MFCCs, deltas, and delta-deltas) from all clips, then writing features to CSVs
# (Not that we are not extracting vowel segments like we do with speech classifiers.)

os.chdir(class_dir_pathname)

for dir_name in [item for item in os.listdir('./') if os.path.isdir(item)]:
    print("> Starting " + dir_name)
    try:
        os.chdir(os.path.join(class_dir_pathname, dir_name))
        try: os.mkdir('./_mfccs_and_deltas')
        except: pass
        filenames = [item for item in os.listdir('./') if item[-4:].lower()=='.wav']
        for filename in filenames:
            csv_out_path = './_mfccs_and_deltas/' + filename[:-4] + '.mfcc.csv'
            if not os.path.isfile(csv_out_path):
                try:
                    mfccs = attk.get_mfccs_and_deltas(filename, n_mfcc=30, n_fft=4096, freq_min=100, freq_max=16000)
                    if len(mfccs) > 0:
                        with open(csv_out_path, 'w') as fo:
                            csv_writer = csv.writer(fo)
                            csv_writer.writerows(mfccs)  
                except Exception as e:
                    print('FILE ERROR: ' + filename)
                    print(e)
    except Exception as e:
        print('SKIPPING DIRECTORY: ' + dir_name)     ## Skipping class directories for which we didn't extract vowels
        #print(e)

In [ ]:
## Continue to the next notebook to train and run the applause classifier.